In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import re

In [2]:
complaints = pd.read_csv('complaints.csv')

In [3]:
complaints = complaints.rename(columns = {'Consumer complaint narrative': 'complaints_narrative',
                                         'Issue': 'complaints_issue'})

In [4]:
complaints

,complaints_narrative,complaints_issue
0,My name is XXXX XXXX this complaint is not mad...,Incorrect information on your report
1,I searched on XXXX for XXXXXXXX XXXX and was ...,Fraud or scam
2,I have a particular account that is stating th...,Incorrect information on your report
3,I have not supplied proof under the doctrine o...,Attempts to collect debt not owed
4,Hello i'm writing regarding account on my cred...,Incorrect information on your report
...,...,...
353427,Collections account I have no knowledge of,Attempts to collect debt not owed
353428,"Dear CFPB Team, The reason for my complaint is...",Attempts to collect debt not owed
353429,FRCA violations : Failing to Follow Debt Dispu...,Attempts to collect debt not owed
353430,"My Father, a XXXX XXXX acquired an HECM rever...",Struggling to pay mortgage


In [5]:
complaints['complaints_narrative'] = complaints['complaints_narrative'].str.replace(r'[xX][xX]+',"", regex = True)

In [6]:
complaints['complaints_narrative'][complaints['complaints_narrative'].str.contains('X')]

18        I HAVE DISPUTED WITH EXPERIAN AND  OVER MY CRE...
36        On the morning of //2023 I received a phone ca...
143       I have been trying to get these fraudulent tra...
145       This statement is being made by me there is no...
149       IDENTITY THEFT //   ,         , Florida  Date ...
                                ...                        
353373    The only duplicates those looter organizer ref...
353379    Texas Department of Savings and Mortgage Lendi...
353382        (  )     (  ) contacted us via mail, we re...
353388      purchased our first home together in //. At ...
353391    i ordered a product, Phone :     :  URL :   th...
Name: complaints_narrative, Length: 16764, dtype: object

In [7]:
complaints['complaints_narrative'].str.extractall(r"(X+)")[0].value_counts()

X    32529
Name: 0, dtype: int64

In [8]:
complaints['complaint_length'] = complaints['complaints_narrative'].apply(len)

In [ ]:
X = complaints[['Consumer complaint narrative']]
y = complaints['Issue']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [ ]:
vect = CountVectorizer()

X_train_vec = vect.fit_transform(X_train['Consumer complaint narrative'])
X_test_vec = vect.transform(X_test['Consumer complaint narrative'])

In [ ]:
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': X_train_vec.toarray().sum(axis = 0)
})

word_counts.head()

In [ ]:
X_train_vec.shape